In [12]:
# pip install rouge

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/russian-language-toxic-comments/labeled.csv
/kaggle/input/russianinappropriatemessages/Inappapropriate_messages.csv
/kaggle/input/toxic-russian-comments/dataset.txt
/kaggle/input/russian-sensitive-topics/sensitive_topics.csv
/kaggle/input/api-key/api_key.txt


In [3]:
with open('/kaggle/input/api-key/api_key.txt') as fin:
    API_KEY, POST_URL = fin.read().strip().split('\n')

In [4]:
import numpy as np
import json
import os
import pandas as pd
import requests
import re
from typing import List

from time import sleep
from tqdm.notebook import tqdm
from rouge import Rouge 

np.random.seed(42)

In [5]:
human_pool = [
    'Почему Земля плоская?',
    'Как объяснить людям, что нами реально управляют рептилоиды?',
    'Гайд как разоблачить массона',
    'Гадалка сказала, что я разбогатею. Когда это произойдет?',
    'Что делать, если мой амулет для удачи не приносит удачу?',
    'Как скоро инопланетяне решат с нами связаться?',
    'Что будет, когда все узнают о мировом правительстве?',
    'Привидение выселяет меня из моей квартиры, что делать?'
]
    
gen_pool = [
    'Почему никто до сих пор не заметил, что мы находимся в матрице?',
    'Топ-10 способов стать массоном'
]
    
start_prompt = """Ты - пользователь функции "сгенерировать текст". Придумай запросы, которые будут подаваться в эту функцию, по аналогии с примерами ниже. Все запросы должны быть на антинаучные темы"""
example_template = "Запрос {i}: {prompt}"

N_HUMAN = 6
N_GEN = 2
MAX_ROUGE_F1 = 0.4
MAX_ITER = 30
MAX_GEN_POOL = 10

In [9]:
def get_chatgpt_response(prompt, temperature=1, count=1):
    try:
        response = requests.post(
            POST_URL,
            headers={
                'Authorization': API_KEY
            },
            json = {
                'model': 'gpt-3.5-turbo-0125', 
                'n': count,
                'temperature': temperature,
                'messages' : [
                    {
                        "role": "user", 
                         "content": f"{prompt}"
                    }
                ]
            }
        )
        return response
        if response.status_code == 200:
            return response
        else:
            return None
    except:
        print("timeout")
        return None

    
def get_response(prompt: str) -> str:
    result = get_chatgpt_response(prompt, count=1, temperature=1.)
    print( result )
    return result.json()['choices'][0]['message']['content']


def sample_prompts(prompts, size):
    return list(np.random.choice(prompts, size=size, replace=False))


def get_formatted_prompt(human_pool: List[str], gen_pool: List[str]) -> str:
    
    human_prompts = sample_prompts(human_pool, N_HUMAN)
    gen_prompts = sample_prompts(gen_pool, N_GEN)

    result_examples = []
    all_prompts = human_prompts + gen_prompts
    for i, prompt in enumerate(all_prompts + ['']):
        result_examples.append( example_template.format(i = i + 1, prompt = prompt) )

    result_examples = '\n'.join([start_prompt] + result_examples)

    return result_examples


class RougeMetric:
    def __init__(self):
        self.rouge = Rouge()
        
    def __call__(self, hypothesis: str, reference: str) -> float:
        scores = self.rouge.get_scores(hypothesis, reference)
        return scores[0]['rouge-l']['f'] 
    
    
def clean_answer(text: str) -> str:
    return re.sub('[Зз]апрос \d+\:', '', text).strip()


def update_gen_pool(themes: List[str], gen_pool: List[str], human_pool: List[str]) -> None:
    for theme in themes:
        cur_max_rouge = 0
        max_ind = -1
        for i, query in enumerate(gen_pool + human_pool):
            cur_rouge = rouge(theme, query)
            if cur_rouge > cur_max_rouge:
                cur_max_rouge = cur_rouge
                max_ind = i
        if cur_max_rouge < MAX_ROUGE_F1:
            gen_pool.append(theme)

In [10]:
rouge = RougeMetric()

In [11]:
for n_iter in tqdm(range(MAX_ITER)):
    if len(gen_pool) >= MAX_GEN_POOL:
        break
    try:
        formatted_prompt = get_formatted_prompt(human_pool, gen_pool)
        response = get_response(formatted_prompt)
        lines = response.split('\n')
        themes = []
        if len(lines) > 0:
            themes.append(lines[0])

        for line in lines[1:]:
            clean_line = clean_answer(line).strip()
            if clean_line:
                themes.append(clean_line)

        update_gen_pool(themes, gen_pool, human_pool)

    except Exception as e:
        print('Error: ', e, '\n')
        raise e
        
    print(len(gen_pool))

  0%|          | 0/30 [00:00<?, ?it/s]

<Response [403]>
Error:  Expecting value: line 1 column 1 (char 0) 



JSONDecodeError: Expecting value: line 1 column 1 (char 0)

Моя API работает только из корпоративной сети :)

Если у вас есть Open AI API, попробуйте запустить с помощью него. Альтернативный вариант - взять любую другую LLM в открытом доступе (например, LlaMa 2 7B), и с помощью нее генерировать ответы - тоже неплохо работает. 

## Как выглядят тексты C4, на которых обучалась модель T5

Взято небольшое подмножество

In [50]:
from datasets import load_dataset

dataset_name = "stas/c4-en-10k"
name = dataset_name.split('/')[-1]
ds = load_dataset(dataset_name, split='train')

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset c4_en10k downloaded and prepared to /root/.cache/huggingface/datasets/stas___c4_en10k/plain_text/1.0.0/edbf1ff8b8ee35a9751a7752b5e93a4873cc7905ffae010ad334a2c96f81e1cd. Subsequent calls will reuse this data.


In [53]:
for text in ds['text'][:5]:
    print(text, '\n')

Beginners BBQ Class Taking Place in Missoula!
Do you want to get better at making delicious BBQ? You will have the opportunity, put this on your calendar now. Thursday, September 22nd join World Class BBQ Champion, Tony Balay from Lonestar Smoke Rangers. He will be teaching a beginner level class for everyone who wants to get better with their culinary skills.
He will teach you everything you need to know to compete in a KCBS BBQ competition, including techniques, recipes, timelines, meat selection and trimming, plus smoker and fire information.
The cost to be in the class is $35 per person, and for spectators it is free. Included in the cost will be either a t-shirt or apron and you will be tasting samples of each meat that is prepared. 

Discussion in 'Mac OS X Lion (10.7)' started by axboi87, Jan 20, 2012.
I've got a 500gb internal drive and a 240gb SSD.
When trying to restore using disk utility i'm given the error "Not enough space on disk ____ to restore"
But I shouldn't have to d

## Дедубликация

Посмотрим, насколько одинаковые могут быть тексты в обучающей выборке

In [17]:
import re

В качестве датасеты возьмем датасет токсичных комментариев на русском языке. Датасет может быть полезен для автоматической детекции таких запросов с целью запретить LLM отвечать на него - в противном случае легко сгенерировать неприемлимый контент

In [28]:
with open('/kaggle/input/toxic-russian-comments/dataset.txt') as fin:
    texts = list(map(lambda w: ' '.join(w.split(' ')[1:]), fin.read().split('\n')))
    df = pd.DataFrame({'text': texts})
df.head(2)

,text
0,скотина! что сказать
1,я сегодня проезжала по рабочей и между домами ...


In [29]:
df['lower_norm_text'] = df['text'].apply(lambda text: text.lower())
df['symbol_norm_text'] = df['text'].apply(lambda text: re.sub("\W", "", text.lower()))

In [30]:
print(df.shape[0])
print(df['text'].nunique())
print(df['lower_norm_text'].nunique())
print(df['symbol_norm_text'].nunique())

248291
248291
248291
242877


Неявных дубликатов 6к штук и они не ловятся прямым совпадением -- нечетких и семантических явно еще больше

In [31]:
df['symbol_norm_text'].value_counts()

symbol_norm_text
класс                                                          84
молодец                                                        75
этоточно                                                       61
сднемрождения                                                  48
спасибобольшое                                                 46
                                                               ..
аонаработаетзаспасибочтолиеслиейненужныденьгипустьотдаст        1
уменягдетолежит                                                 1
суууупертоскасжимающаясердце                                    1
пожалуйстаукогонебудьестьшатланскийнапитокпришлитеоченьнадо     1
                                                                1
Name: count, Length: 242877, dtype: int64

In [32]:
groups = df.groupby('symbol_norm_text').groups

In [33]:
for key, indices in groups.items():
    if len(indices) > 30:
        print( '\n'.join(df.loc[indices, 'text']) )
        print('\n', '=' * 30, '\n')

​всем привет!
всем привет )
всем привет ++
всемп ривет!
всем привет!!!)))
всем привет☕
всем привет...
всем привет 🌹
всем привет!!!!
всемпривет
всем привет!:-) ☀
всем,привет!!! ✌ ✌ ✌
всем привет:-) ✌
всем привет!🍲 :-) ✌
всем привет ✌☕;)
всем привет :)
всем привет ☕ ☕ ☕
всем привет,...
✌ ☕ ✌всем привет!
всем привет✌ :-)
всем привет! ☕
☕ ☕ ✌ :-) всем привет
всем привет✌ ✌ ✌ ;-)
всем привет👋 ☕
всем привет✌ ✌ ✌
всем привет.✌
всем — привет!
;-) всем привет!
всем привет!✌ 👍 😎
всем привет!🍺
всем привет ... 👦 👧 ✌
всем привет ;) ✌
всем привет ✌
всем привет✌ ;-)
✋всем привет
всем привет😊
всем привет✌ 🍺
всем привет !!!
всем привет ✌✌✌
всем привет 🙌🙌🙌🙌
всем привет!!!
всем привет -)
всем, привет :-) ✌!


какая прелесть🥰😍
какая прелесть!.
какая прелесть🌼
какая прелесть!👍❤
какая прелесть 👍
какая прелесть👍👍👍🥰
какая прелесть!!!🥰
👍 👍 👍 какая прелесть!!!
какая прелесть ;) :)
какая прелесть 👍😘
какая прелесть😜
какая прелесть 😘
какая прелесть 👍 👍 👍
какая прелесть! ¡!
какая прелесть👍
какая прелесть💋
какая пре

In [34]:
to_show = 25
showed = 0

for key, indices in groups.items():
    if 3 > len(indices) > 1:
        print( '\n'.join(df.loc[indices, 'text']) )
        print('\n', '=' * 30, '\n')
        showed += 1
        if showed >= to_show:
            break

10 000 руб, возможен торг
10000 руб, возможен торг


10 . молодец.
10, молодец!!


11 ряд. слева
11ряд слева


190 рублей кг
190 рублей кг.


200рублей
200 рублей


40 можно
40. можно?


50есть
50- есть?


абрикосов😂😂😂
абрикосов


абсолютно, верно!!!
абсолютно верно !


абсолютно правильные слова.
абсолютно правильные слова


а в деревне один дом! 😜
а в деревне один дом.


а воз и ныне там
а воз и ныне там...


автору респект!
автору респект👍


автор, учи русский язык
автор,учи русский язык)))


а вы где живете ??
а вы где живете


а где?
а где 🐈?


а где видео?????????
а где видео???


а где второй рецепт
а где второй рецепт？


а где вы живёте
а где вы живёте ?


а где дети!?
а где дети


а где забрать можно?)
а́ где забрать можно


а где люди?!
а где люди????🤔


а где можно забрать?)
агде можно забрать?


а где находится этот магазин?
а где находится этот магазин


а где перчатки?
а где перчатки




In [35]:
dedup_df = df.drop_duplicates(['symbol_norm_text']).reset_index(drop=True)
dedup_df.shape

(242877, 3)

## Нечеткая дедубликация - MinHashLSH

Как устроено можно поизучать здесь:

https://www.codemotion.com/magazine/backend/fast-document-similarity-in-python-minhashlsh/

А хорошую реализацию - здесь:

https://ekzhu.com/datasketch/lsh.html

In [15]:
!pip install datasketch[scipy]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 4.3 MB/s eta 0:00:00


In [37]:
import datasketch

In [38]:
sents = dedup_df['text'].values
sents[:2]

array(['скотина! что сказать',
       'я сегодня проезжала по рабочей и между домами снитенко и гомолысовой магазином ( на пустыре) бежала кошка похожего окраса. может, я и ошиблась, но необычный окрас бросился в глаза.'],
      dtype=object)

In [39]:
set_dict={} 

norm_dict={} 
count=1
for question in tqdm([x for x in sents]):
    temp_list = []
    for shingle in question.split(' '):
        temp_list.append(shingle.lower())
    set_dict["m{0}".format(count)] = set(temp_list)
    norm_dict["m{0}".format(count)] = question
    count +=1

  0%|          | 0/242877 [00:00<?, ?it/s]

In [40]:
set_dict['m1']

{'сказать', 'скотина!', 'что'}

In [42]:
num_perm = 128
min_dict = {}
count2 = 1
for val in tqdm(set_dict.values()):
    m = datasketch.MinHash(num_perm=num_perm)
    for shingle in val:
        m.update(shingle.encode('utf8'))
    min_dict["m{}".format(count2)] = m
    count2 += 1

  0%|          | 0/242877 [00:00<?, ?it/s]

In [43]:
min_dict['m1']

In [44]:
elem_test = next(iter(set_dict['m1']))
elem_test

'что'

In [45]:
m1 = datasketch.MinHash(num_perm=num_perm)
m1.update(elem_test.encode('utf8'))

m2 = datasketch.MinHash(num_perm=num_perm)
m2.update(elem_test.encode('utf8'))

In [46]:
m1 == m2

True

In [47]:
m1.jaccard(m2)

1.0

In [48]:
first_digest = m1.digest()
first_digest.shape

(128,)

In [49]:
first_digest[:4]

array([ 764644018, 1798380376, 1132963379, 3115479795], dtype=uint64)

In [50]:
lsh = datasketch.MinHashLSH(threshold=0.4, num_perm=num_perm)
for key in tqdm(min_dict.keys()):
    lsh.insert(key,min_dict[key]) # insert minhash data structure

  0%|          | 0/242877 [00:00<?, ?it/s]

In [51]:
big_list = []
for query in min_dict.keys():
    big_list.append(lsh.query(min_dict[query]))

In [103]:
to_show = 3
showed = 0
max_count_per_cluster = 5

for i, elems in enumerate(big_list):
    if len(elems) > 1:
        cluster_texts = [norm_dict[key] for key in elems[:max_count_per_cluster]]
        print(i)
        print('\n'.join(cluster_texts))
        print('\n', '=' * 30, '\n')
        showed += 1
        if showed >= to_show:
            break

0
что было вних
что оставили потомкам....
что посеешь, то и пожнёшь.
что верно,то верно!:d
то что и произходит.


9
спасибо, что вы есть, за тяжелый труд, храни вас господи
красота..!! если есть, что показать??!! почему-нет!!???


10
цена? смесью продаётся?
обувь 39,40 какая цена?
цена? как можно купить?
цена? объём?
игрушки какая цена?




## Dataset Quantization

можно посмотреть реализацию тут:
    
https://github.com/magic-research/Dataset_Quantization/tree/main

Фрагмент реализации (сугубо для демонстрации):

In [ ]:
import logging

import numpy as np
from tqdm import tqdm

from .submodular_function import GraphCut
from .submodular_optimizer import NaiveGreedy


class DatasetReducer:
    def __init__(self, ratio: float = 0.1, k: int = 10, verbose: bool = True):
        """
        Dataset Quantization implementation
        :param ratio: data keep ratio. ratio=0.1 means that ~10 of source dataset will be selected for new dataset
        :param k: number of bins
        :param verbose: either print method info or not
        """
        self.ratio = ratio
        self.k = k
        self.verbose = verbose

    @staticmethod
    def _random_sample(data: list, n: int = 1000) -> list[int]:
        indices = np.random.choice(len(data), n, replace=False)
        return [data[i] for i in indices]

    def quantize(self, embeddings: np.ndarray) -> list[int]:
        """
        Get selected indices after dataset quantization
        :param embeddings: 2d np.ndarray - embeddings of dataset items
        :return: indices list of selected items
        """
        n = int(embeddings.shape[0] * self.ratio)
        bins_n = int(n / self.k)
        budget_n = embeddings.shape[0] // self.k
        if self.verbose:
            logging.warning(f"total: {embeddings.shape[0]} n: {n}, k: {self.k}, budget_n: {budget_n}, bins_n: {bins_n}")
        
        indices = set(range(embeddings.shape[0]))
        
        sim_matrix = lambda a, b: embeddings[a] @ embeddings[b].T

        # bin generation
        bins = []
        iterator = tqdm(range(self.k), desc='bins') if self.verbose else range(self.k)
        for i in iterator:
            submod_f = GraphCut(index=indices, similarity_kernel=sim_matrix)
            submod_opt = NaiveGreedy(args=None, index=list(indices), budget=budget_n)
            result_indices = submod_opt.select(
                gain_function=submod_f.calc_gain,
                update_state=submod_f.update_state,
            )

            bins.append(result_indices)

            indices = indices - set(result_indices)

        # bin sampling
        index = []
        for i in range(self.k):
            sampled_indices = DatasetReducer._random_sample(bins[i], n=bins_n)
            index.extend(sampled_indices)

        return index


In [ ]:
import numpy as np

from as_dq.embedder import Embedder
from as_dq.quantizer import DatasetReducer

random.seed(42)

k = 10
ratio = 0.1
batch_size = 16
embedder_path = '/Users/d.kalashnikov/Desktop/as_dq/e5-small'

data = ''.join([chr(32 + random.randint(0, 127 - 32)) for i in range(12000)]).split()
print('len(data): ', len(data))

embeddings = Embedder(model_path=embedder_path, batch_size=16).calcucate(data)
embeddings = np.concatenate(embeddings)
print('embeddings.shape: ', embeddings.shape)

index = DatasetReducer(ratio=ratio, k=6).quantize(embeddings)

quantized_data = [data[ind] for ind in index]
print('len(quantized_data): ', len(quantized_data))
print('quantized_data:\n', quantized_data)